In [1]:
import pyspark 
from pyspark.sql import SparkSession 
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [ ]:
path_to_google_creds = "path/to/creds.json"

In [2]:
conf = SparkConf() \
.setMaster('local[*]') \
.setAppName('test') \
.set("spark.jars", "./lib/gcs-connector-hadoop3-2.2.5.jar") \
.set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
.set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", path_to_google_creds) 

In [3]:
sc = SparkContext(conf=conf) 
sc._jsc.hadoopConfiguration().set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS") 
sc._jsc.hadoopConfiguration().set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") 
sc._jsc.hadoopConfiguration().set("fs.gs.auth.service.account.json.keyfile", path_to_google_creds) 
sc._jsc.hadoopConfiguration().set("fs.gs.auth.service.account.enable", "true")

24/04/14 15:29:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
# sc.stop()

In [4]:
spark = SparkSession.builder \
.config(conf=sc.getConf()) \
.getOrCreate()

In [5]:
BUCKET_NAME = <Fill in bucket name>
df_transaction = spark.read.parquet(f"gs://{BUCKET_NAME}/raw/eth/transactions/date=2024-04-08/")

In [8]:
from pyspark.sql import types

In [14]:
df_transaction.schema

StructType([StructField('gas', LongType(), True), StructField('hash', StringType(), True), StructField('input', StringType(), True), StructField('nonce', LongType(), True), StructField('value', DoubleType(), True), StructField('block_number', LongType(), True), StructField('block_hash', StringType(), True), StructField('transaction_index', LongType(), True), StructField('from_address', StringType(), True), StructField('to_address', StringType(), True), StructField('gas_price', LongType(), True), StructField('receipt_cumulative_gas_used', LongType(), True), StructField('receipt_gas_used', LongType(), True), StructField('receipt_contract_address', StringType(), True), StructField('receipt_status', LongType(), True), StructField('receipt_effective_gas_price', LongType(), True), StructField('transaction_type', LongType(), True), StructField('max_fee_per_gas', LongType(), True), StructField('max_priority_fee_per_gas', LongType(), True), StructField('block_timestamp', TimestampType(), True),

In [18]:
transaction_schema = types.StructType(
    [
        types.StructField('gas', types.LongType(), True), 
        types.StructField('hash', types.StringType(), True), 
        types.StructField('input', types.StringType(), True), 
        types.StructField('nonce', types.LongType(), True), 
        types.StructField('value', types.DoubleType(), True), 
        types.StructField('block_number', types.LongType(), True), 
        types.StructField('block_hash', types.StringType(), True), 
        types.StructField('transaction_index', types.LongType(), True), 
        types.StructField('from_address', types.StringType(), True), 
        types.StructField('to_address', types.StringType(), True), 
        types.StructField('gas_price', types.LongType(), True), 
        types.StructField('receipt_cumulative_gas_used', types.LongType(), True), 
        types.StructField('receipt_gas_used', types.LongType(), True), 
        types.StructField('receipt_contract_address', types.StringType(), True), 
        types.StructField('receipt_status', types.LongType(), True), 
        types.StructField('receipt_effective_gas_price', types.LongType(), True), 
        types.StructField('transaction_type', types.LongType(), True), 
        types.StructField('max_fee_per_gas', types.LongType(), True), 
        types.StructField('max_priority_fee_per_gas', types.LongType(), True), 
        types.StructField('block_timestamp', types.TimestampType(), True), 
        types.StructField('date', types.DateType(), True), 
        types.StructField('last_modified', types.TimestampType(), True)
    ]
)

In [19]:
df_transaction = spark.read.schema(transaction_schema).parquet(f"gs://{BUCKET_NAME}/raw/eth/transactions/date=2024-04-08/")

In [20]:
df_transaction

DataFrame[gas: bigint, hash: string, input: string, nonce: bigint, value: double, block_number: bigint, block_hash: string, transaction_index: bigint, from_address: string, to_address: string, gas_price: bigint, receipt_cumulative_gas_used: bigint, receipt_gas_used: bigint, receipt_contract_address: string, receipt_status: bigint, receipt_effective_gas_price: bigint, transaction_type: bigint, max_fee_per_gas: bigint, max_priority_fee_per_gas: bigint, block_timestamp: timestamp, date: date, last_modified: timestamp]

In [21]:
df_transaction.show()

24/04/14 15:50:17 ERROR Executor: Exception in task 0.0 in stage 2.0 (TID 2)
org.apache.spark.sql.execution.QueryExecutionException: Parquet column cannot be converted in file gs://ethereum_etl_datalake/raw/eth/transactions/date=2024-04-08/part-00000-4c2fde61-a460-4d0e-bfc1-07acff938569-c000.snappy.parquet. Column: [date], Expected: date, Found: BINARY
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:278)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:561)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.columnartorow_nextBatch_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCod

Py4JJavaError: An error occurred while calling o156.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2) (de-zoomcamp.us-central1-c.c.quantum-fusion-417707.internal executor driver): org.apache.spark.sql.execution.QueryExecutionException: Parquet column cannot be converted in file gs://ethereum_etl_datalake/raw/eth/transactions/date=2024-04-08/part-00000-4c2fde61-a460-4d0e-bfc1-07acff938569-c000.snappy.parquet. Column: [date], Expected: date, Found: BINARY
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:278)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:5

In [16]:
import pandas as pd
# df_yellow_pd = pd.read_parquet('data/raw/yellow/2023/01/').iloc[:1000]
df_yellow_pd = pd.read_parquet("raw/eth/transactions/date=2024-04-08/")

In [17]:
df_yellow_pd.dtypes

gas                                     int64
hash                                   object
input                                  object
nonce                                   int64
value                                 float64
block_number                            int64
block_hash                             object
transaction_index                       int64
from_address                           object
to_address                             object
gas_price                               int64
receipt_cumulative_gas_used             int64
receipt_gas_used                        int64
receipt_contract_address               object
receipt_status                          int64
receipt_effective_gas_price             int64
transaction_type                        int64
max_fee_per_gas                         int64
max_priority_fee_per_gas                int64
block_timestamp                datetime64[ns]
date                                   object
last_modified                  dat